In [203]:
from pyspark.mllib.linalg import DenseVector
from pyspark.mllib.random import RandomRDDs

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col

import gc
import os
import pandas as pd
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

spark.conf.set("spark.sql.caseSensitive", "true")

In [221]:
df = spark.read.json(r'C:\Users\N716307\Downloads\TweetCount-master\TweetCount-master\*.gz')
df.registerTempTable("df")
df.printSchema()


root
 |-- request: struct (nullable = true)
 |    |-- documentReferer: string (nullable = true)
 |    |-- error: string (nullable = true)
 |    |-- eventId: string (nullable = true)
 |    |-- frameId: long (nullable = true)
 |    |-- fromCache: boolean (nullable = true)
 |    |-- ip: string (nullable = true)
 |    |-- is_online: boolean (nullable = true)
 |    |-- mainFrameRequestId: string (nullable = true)
 |    |-- method: string (nullable = true)
 |    |-- navigationId: long (nullable = true)
 |    |-- openerTabId: long (nullable = true)
 |    |-- parentFrameId: long (nullable = true)
 |    |-- redirectUrl: string (nullable = true)
 |    |-- requestHeaders: struct (nullable = true)
 |    |    |-- Accept: string (nullable = true)
 |    |    |-- Accept-Encoding: string (nullable = true)
 |    |    |-- Accept-Language: string (nullable = true)
 |    |    |-- Avail-Dictionary: string (nullable = true)
 |    |    |-- Content-Type: string (nullable = true)
 |    |    |-- DNT: string (nul

In [227]:
dg = sqlContext.sql("""
    SELECT request.*,request.requestHeaders.*,request.responseHeaders.*,server_request.*,server_request.user_map.*,
    request.requestHeaders.`Content-Type` as requestHeaders_content_type,request.responseHeaders.`Content-Type` as responseHeader_content_type,
    request.requestHeaders.Origin as requestHeaders_Origin,request.responseHeaders.Origin as responseHeader_Origin,
    request.requestHeaders.Referer as requestHeaders_Referer,request.responseHeaders.Referer as responseHeader_Referer,
    request.requestHeaders.`User-Agent` as requestHeaders_User_Agent,request.responseHeaders.`User-Agent` as responseHeader_User_Agent
    FROM df
""").drop('requestHeaders').drop('responseHeaders').drop('server_request').drop('user_map').drop('Content-Type').drop('Origin').drop('Referer').drop('User-Agent')



dg.printSchema()
print('Number of rows: {:d} \nNumber of columns: {:d}'.format(dg.count(),len(dg.columns)))

root
 |-- documentReferer: string (nullable = true)
 |-- error: string (nullable = true)
 |-- eventId: string (nullable = true)
 |-- frameId: long (nullable = true)
 |-- fromCache: boolean (nullable = true)
 |-- ip: string (nullable = true)
 |-- is_online: boolean (nullable = true)
 |-- mainFrameRequestId: string (nullable = true)
 |-- method: string (nullable = true)
 |-- navigationId: long (nullable = true)
 |-- openerTabId: long (nullable = true)
 |-- parentFrameId: long (nullable = true)
 |-- redirectUrl: string (nullable = true)
 |-- requestId: string (nullable = true)
 |-- requestType: string (nullable = true)
 |-- statusCode: long (nullable = true)
 |-- statusLine: string (nullable = true)
 |-- tabId: long (nullable = true)
 |-- timeStamp: double (nullable = true)
 |-- type: string (nullable = true)
 |-- url: string (nullable = true)
 |-- windowName: string (nullable = true)
 |-- windowTitle: string (nullable = true)
 |-- Accept: string (nullable = true)
 |-- Accept-Encoding: st

In [275]:
dg.select('method').distinct().show()

+------+
|method|
+------+
|  POST|
|  null|
|   GET|
+------+



In [241]:
c = dg.count()
null_drop = []
distinct_high = []
for X,y in dg.dtypes:
    x = '`' + X + '`'
    null_count = dg.filter(col(x).isNull()).count()
    if y == 'string':
        distinct_count = dg.select(x).distinct().count()
        if distinct_count > 100:
            distinct_high.append(x)
    else:
        distinct_count = 0
    if null_count*1./c > 0.2:
        null_drop.append(x)
    print('{:40s} {:8d} ({:5.2f}%), {:8d}'.format(X,null_count,null_count*100./c,distinct_count))

`documentReferer`             :     2255 (42.48%),      341
`error`                       :     5298 (99.79%),        2
`eventId`                     :      763 (14.37%),     4386
`frameId`                     :        3 ( 0.06%),        0
`fromCache`                   :        3 ( 0.06%),        0
`ip`                          :      171 ( 3.22%),      933
`is_online`                   :     5298 (99.79%),        0
`mainFrameRequestId`          :     5306 (99.94%),        3
`method`                      :        3 ( 0.06%),        3
`navigationId`                :      810 (15.26%),        0
`openerTabId`                 :     5305 (99.92%),        0
`parentFrameId`               :        3 ( 0.06%),        0
`redirectUrl`                 :     4551 (85.72%),      560
`requestId`                   :     4549 (85.68%),      685
`requestType`                 :        0 ( 0.00%),        1
`statusCode`                  :       14 ( 0.26%),        0
`statusLine`                  :       14

In [274]:
dg.select('`method`').filter(dg['`method`']!='').show(100,False)

+------+
|method|
+------+
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
|GET   |
+------+
only showing top 100 rows



In [267]:
keep_cols = []
for x in dg.columns:
    x = '`' + x + '`'
    if x not in null_drop:
        keep_cols.append(x)
        
dg = dg.select(keep_cols)

In [268]:
dg.columns

['eventId',
 'frameId',
 'fromCache',
 'ip',
 'method',
 'navigationId',
 'parentFrameId',
 'requestType',
 'statusCode',
 'statusLine',
 'tabId',
 'timeStamp',
 'type',
 'url',
 'windowName',
 'windowTitle',
 'Accept',
 'Accept-Encoding',
 'Accept-Language',
 'Upgrade-Insecure-Requests',
 'accept_language',
 'country_code',
 'partner_id',
 'request_unixtime',
 'software_id',
 'user_agent',
 'user_guid',
 'x_forwarded_for',
 'requestHeaders_Referer',
 'requestHeaders_User_Agent']

In [57]:
from pyspark.sql import types as t

dg = dg.withColumn("TimestampType",F.to_timestamp(dg["request_unixtime"]))

In [58]:
def rdd_map_t_row(one_row,model_lr):

    # one_row is of type lines type=<class 'pyspark.sql.types.Row'>
    row_dict = one_row.asDict()
    pd_df = pd.DataFrame([row_dict]) # has to be a list object
    print("ONE:ROW:now with PANDAS DF={}".format(pd_df))
    try:
        y_pred = model_lr.predict(pd_df)
        print("ONE:ROW:PD_DF_Y_PRED={}".format(y_pred))
    except:
        print("error:MODEL:PREDICT")

    ret_row = one_row # access the elements like one_row.col1, one_row.col2  where col1 is the name of the column
#     try:
#         # cannto do spark.createDataFrame(pd_df)  because we have to return the rdd.row object
#         ret_pd_dict = pd_df.to_dict(orient='records')
#         print("ONE:ROW:PD:DICT=type={0}::dict={1}".format(type(ret_pd_dict),ret_pd_dict))
#         ret_pd_dict = ret_pd_dict[1] # get the SECOND because first has header --first and the only element we need since we create 1 row
#         ret_row = Row(**ret_pd_dict)
#     except:
#         print("error:SPARK:PD:TO:DF")
    ret_pd_dict = pd_df.to_dict(orient='records')
    print("ONE:ROW:PD:DICT=type={0}::dict={1}".format(type(ret_pd_dict),ret_pd_dict))
    print(ret_pd_dict)
    ret_pd_dict = ret_pd_dict[0] # get the SECOND because first has header --first and the only element we need since we create 1 row
#     ret_row = Row(**ret_pd_dict)
    return ret_row

In [47]:
new_rdd = df.rdd.map(lambda x: rdd_map_t_row(x,model_lr))

In [59]:
x = rdd_map_t_row(df.rdd.first(),model_lr)

ONE:ROW:now with PANDAS DF=         _1        _2        _3        _4
0  0.851571  0.124906  0.667121  0.354662
ONE:ROW:PD_DF_Y_PRED=[9.56496148]
ONE:ROW:PD:DICT=type=<class 'list'>::dict=[{'_1': 0.8515712526498084, '_2': 0.12490555818176752, '_3': 0.6671208952627372, '_4': 0.35466243926679875}]
[{'_1': 0.8515712526498084, '_2': 0.12490555818176752, '_3': 0.6671208952627372, '_4': 0.35466243926679875}]


In [60]:
x

Row(_1=0.8515712526498084, _2=0.12490555818176752, _3=0.6671208952627372, _4=0.35466243926679875)

In [49]:
new_rdd.first()

Row(_1=0.8515712526498084, _2=0.12490555818176752, _3=0.6671208952627372, _4=0.35466243926679875)

In [42]:
import numpy as np
from sklearn.linear_model import LinearRegression
X = np.array([[1, 1,1,1], [1, 2,2,3], [2, 2,4,5], [2, 3,6,7]])
# y = 1 * x_0 + 2 * x_1 + 3
y = np.dot(X, np.array([1, 2,5,6])) + 3
model_lr = LinearRegression().fit(X, y)

In [43]:
model_lr.predict(pandas_df)

array([ 9.56496148,  7.85476856,  8.15746101, 14.09041434, 10.36096284,
        9.17911023,  6.96177506, 11.24547773])